In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [3]:
nc = pd.read_csv("/content/drive/MyDrive/[b.a.f]/news_cos_stock_nc.csv")
nc

,Date,Open,High,Low,Close,Volume,Change,5MA,5Disparity,20MA,...,Institutional_trading,Individual_trading,Foreign_trading,Short_volume,Short_left,Cos_sim,wil,buy,Fluctuation,label
0,2019-03-04,456500,459000,442000,448000,146351,-0.028200,446800.0,1.002686,449600.0,...,8476,23314,-31924,11040,250525,0,458000.0,0,0,0.200000
1,2019-03-05,449500,450500,443000,445000,66258,-0.006696,449300.0,0.990430,448475.0,...,-7912,6665,1252,2638,248958,1,458000.0,0,1,0.142857
2,2019-03-06,445000,451000,437000,449500,72925,0.010112,450600.0,0.997559,447525.0,...,2828,-1659,-1151,728,244957,1,448750.0,1,0,0.368421
3,2019-03-07,452000,454000,435500,440000,79647,-0.021135,448700.0,0.980611,445825.0,...,-11852,17774,-5776,1006,242707,0,459000.0,0,0,0.428571
4,2019-03-08,437500,439500,429000,430500,70249,-0.021591,442600.0,0.972662,444225.0,...,-18064,20081,-4289,2531,239731,0,446750.0,0,1,-0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,2024-02-23,205500,205500,202000,205000,60411,0.004902,207700.0,0.987000,204090.0,...,-417,-2326,2787,103,5793,1,208750.0,0,0,-0.333333
1231,2024-02-26,206000,207000,201000,201500,67156,-0.017073,205400.0,0.981013,204535.0,...,-9120,601,8741,27,5793,1,207750.0,0,0,-0.333333
1232,2024-02-27,201000,202000,196500,198100,106971,-0.016873,203420.0,0.973847,204750.0,...,-2072,21354,-19338,111,5793,1,204000.0,0,1,-0.333333
1233,2024-02-28,198200,201000,198000,199600,49470,0.007572,201640.0,0.989883,204980.0,...,5587,5136,-10732,20,5793,1,200950.0,1,0,-0.333333


In [4]:
nc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1235 entries, 0 to 1234
Data columns (total 43 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   1235 non-null   object 
 1   Open                   1235 non-null   int64  
 2   High                   1235 non-null   int64  
 3   Low                    1235 non-null   int64  
 4   Close                  1235 non-null   int64  
 5   Volume                 1235 non-null   int64  
 6   Change                 1235 non-null   float64
 7   5MA                    1235 non-null   float64
 8   5Disparity             1235 non-null   float64
 9   20MA                   1235 non-null   float64
 10  20Disparity            1235 non-null   float64
 11  60MA                   1235 non-null   float64
 12  60Disparity            1235 non-null   float64
 13  120MA                  1235 non-null   float64
 14  120Disparity           1235 non-null   float64
 15  bol_

In [5]:
# 타겟레이블인 change의 값의 종류 확인
print(nc['Change'].value_counts())

Change
 0.000000    53
-0.002433     3
-0.003000     3
 0.001855     3
-0.001215     3
             ..
-0.018293     1
 0.013665     1
 0.007353     1
-0.014599     1
-0.028056     1
Name: count, Length: 1130, dtype: int64


In [6]:
# 범주형 변수
print(nc['Cos_sim'].unique())
print(nc['buy'].unique())
print(nc['Fluctuation'].unique())

[0 1]
[0 1]
[0 1]


In [7]:
# 타겟변수 이진분류로 변경
df = nc.copy()
df['change_label'] = (df['Change'] >= 0).astype(int)
df

,Date,Open,High,Low,Close,Volume,Change,5MA,5Disparity,20MA,...,Individual_trading,Foreign_trading,Short_volume,Short_left,Cos_sim,wil,buy,Fluctuation,label,change_label
0,2019-03-04,456500,459000,442000,448000,146351,-0.028200,446800.0,1.002686,449600.0,...,23314,-31924,11040,250525,0,458000.0,0,0,0.200000,0
1,2019-03-05,449500,450500,443000,445000,66258,-0.006696,449300.0,0.990430,448475.0,...,6665,1252,2638,248958,1,458000.0,0,1,0.142857,0
2,2019-03-06,445000,451000,437000,449500,72925,0.010112,450600.0,0.997559,447525.0,...,-1659,-1151,728,244957,1,448750.0,1,0,0.368421,1
3,2019-03-07,452000,454000,435500,440000,79647,-0.021135,448700.0,0.980611,445825.0,...,17774,-5776,1006,242707,0,459000.0,0,0,0.428571,0
4,2019-03-08,437500,439500,429000,430500,70249,-0.021591,442600.0,0.972662,444225.0,...,20081,-4289,2531,239731,0,446750.0,0,1,-0.166667,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,2024-02-23,205500,205500,202000,205000,60411,0.004902,207700.0,0.987000,204090.0,...,-2326,2787,103,5793,1,208750.0,0,0,-0.333333,1
1231,2024-02-26,206000,207000,201000,201500,67156,-0.017073,205400.0,0.981013,204535.0,...,601,8741,27,5793,1,207750.0,0,0,-0.333333,0
1232,2024-02-27,201000,202000,196500,198100,106971,-0.016873,203420.0,0.973847,204750.0,...,21354,-19338,111,5793,1,204000.0,0,1,-0.333333,0
1233,2024-02-28,198200,201000,198000,199600,49470,0.007572,201640.0,0.989883,204980.0,...,5136,-10732,20,5793,1,200950.0,1,0,-0.333333,1


In [8]:
# 객체 추출
X = df.drop(columns=['Date','Change','change_label'])
y = df['change_label']

In [9]:
#X
y

0       0
1       0
2       1
3       0
4       0
       ..
1230    1
1231    0
1232    0
1233    1
1234    0
Name: change_label, Length: 1235, dtype: int64

In [10]:
# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
x_train

,Open,High,Low,Close,Volume,5MA,5Disparity,20MA,20Disparity,60MA,...,Institutional_trading,Individual_trading,Foreign_trading,Short_volume,Short_left,Cos_sim,wil,buy,Fluctuation,label
869,368500,384000,368000,377500,80910,374600.0,1.007742,378400.0,0.997622,380816.666667,...,889,-29786,28900,5911,132803,1,376750.0,1,0,1.000000
728,551000,565000,546000,550000,251453,540000.0,1.018519,564500.0,0.974314,644666.666667,...,5481,8446,-14755,22344,185311,1,557000.0,1,0,0.275862
803,458500,458500,451000,455000,193455,452200.0,1.006192,432750.0,1.051415,443216.666667,...,-11054,4113,6818,48376,71354,1,461750.0,0,1,0.500000
644,571000,585000,566000,578000,101743,573400.0,1.008022,589300.0,0.980825,716216.666667,...,-5070,-10579,10803,2661,217252,1,581000.0,1,0,0.030303
813,417000,421500,400500,401000,122400,426500.0,0.940211,443875.0,0.903407,442075.000000,...,-10394,19162,-9980,15498,65247,0,428000.0,0,0,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,380000,393000,378500,388000,131707,373500.0,1.038822,376150.0,1.031503,385566.666667,...,48851,-23754,-27648,18392,32857,1,383250.0,1,1,0.636364
1095,276500,279000,272000,274000,107472,276700.0,0.990242,280600.0,0.976479,311808.333333,...,-26476,10553,15591,2378,31108,0,279000.0,0,1,0.000000
1130,233500,234000,226500,230500,86931,242800.0,0.949341,248300.0,0.928313,265833.333333,...,-1719,8178,-5783,5909,150826,1,238750.0,0,0,0.111111
860,369000,374500,363500,363500,72891,375100.0,0.969075,379975.0,0.956642,393483.333333,...,-15610,12310,3255,10935,121300,1,373250.0,1,0,0.657895


In [12]:
# train 비율 확인
print(y_train.value_counts())

change_label
0    495
1    493
Name: count, dtype: int64


In [13]:
# xgboost 훈련
model = XGBClassifier(objective='binary:logistic', use_label_encoder=False, eval_metric='logloss')
model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [14]:
# 예측
y_pred = model.predict(x_test)
y_pred

array([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0])

In [18]:
# 예측2
y_pred_proba = model.predict_proba(x_test)[:,1]
y_pred_proba

array([9.52441454e-01, 1.14733753e-04, 3.68297398e-01, 1.54408038e-01,
       4.49286355e-03, 1.51200080e-03, 3.52400425e-03, 6.43329287e-04,
       8.05322051e-01, 1.65920004e-01, 6.91440344e-01, 7.57561123e-04,
       1.69658312e-03, 8.42153877e-02, 9.99908686e-01, 4.52136941e-04,
       1.19424881e-02, 9.97719944e-01, 5.31710051e-02, 9.83818948e-01,
       9.56893206e-01, 2.07886286e-02, 1.52735375e-02, 2.55973246e-02,
       9.95864987e-01, 2.19603300e-01, 6.04165316e-01, 9.99940753e-01,
       7.42822513e-03, 7.34970439e-03, 4.70076874e-02, 1.13417886e-01,
       3.65504122e-04, 6.88720611e-04, 9.31112647e-01, 2.83056319e-01,
       9.93363738e-01, 9.95522857e-01, 2.16458249e-03, 9.99793708e-01,
       9.89517868e-01, 9.98263419e-01, 4.64870268e-03, 7.13479996e-01,
       5.40510237e-01, 8.72109830e-01, 2.28475302e-01, 7.74883153e-03,
       9.98593509e-01, 1.99505854e-02, 5.35158382e-04, 1.30845746e-03,
       5.57807535e-02, 1.05383154e-02, 2.69844145e-01, 9.96490300e-01,
      

In [15]:
# 평가
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:{:.2f}".format(accuracy))

Accuracy:0.84


In [16]:
# 분류 리포트
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.87      0.84       118
           1       0.87      0.81      0.84       129

    accuracy                           0.84       247
   macro avg       0.84      0.84      0.84       247
weighted avg       0.84      0.84      0.84       247



In [33]:
def get_clf_eval(y_test, pred, pred_proba):
  from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, precision_score, recall_score
  confusion = confusion_matrix(y_test, pred)
  accuracy = accuracy_score(y_test, pred)
  precision = precision_score(y_test, pred)
  recall = recall_score(y_test, pred)
  f1 = f1_score(y_test, pred)
  print('오차 행렬')
  print(confusion)
  print('정확도:{0:.4f}, 정밀도:{1:.4f}, 재현율:{2:.4f}, f1:{3:.4f}'.format(accuracy, precision, recall, f1))

  pred_proba = model.predict_proba(x_test)

In [34]:
get_clf_eval(y_test, y_pred, y_pred_proba)

오차 행렬
[[103  15]
 [ 25 104]]
정확도:0.8381, 정밀도:0.8739, 재현율:0.8062, f1:0.8387
